In [1]:
import geojson
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('dpt2020.csv', sep=';')

df = df[df['dpt'] != 'XX']
df = df[df['dpt'] != '971']
df = df[df['dpt'] != '972']
df = df[df['dpt'] != '973']
df = df[df['dpt'] != '974']

df['sexe'] = df['sexe'].apply(str)

df.loc[df['sexe'] == '1', 'sexe'] = 'M'
df.loc[df['sexe'] == '2', 'sexe'] = 'F'

df = df[df['preusuel'] != '_PRENOMS_RARES']

df['annais'] = df['annais'].astype(int)

df.head()

,sexe,preusuel,annais,dpt,nombre
10885,M,AADIL,1983,84,3
10886,M,AADIL,1992,92,3
10888,M,AAHIL,2016,95,3
10892,M,AARON,1962,75,3
10893,M,AARON,1976,75,3


In [3]:
min_year, max_year = df['annais'].min(), df['annais'].max()
marks_year = list(range(df['annais'].min(), df['annais'].max() + 1, 10))
marks_year = {i:str(i) for i in marks_year}

In [4]:
with open('Departements.geojson') as f:
    geojson_departments = geojson.load(f)

for idx, department in enumerate(geojson_departments['features']):
    department['id'] = department['properties']['code']

In [5]:
def df_filter_gendered_names(df, gendered_names):
    tuples_in_df = pd.MultiIndex.from_frame(df[['preusuel', 'sexe']])
    df = df[tuples_in_df.isin(gendered_names)]
    return df

In [6]:
def df_filter_partial_name(df, partial_name):
    df = df[df['preusuel'].str.startswith(partial_name)]
    return df

In [7]:
def df_filter_names(df, names):
    df = df[df['preusuel'].isin(names)]
    return df

In [8]:
def df_filter_genders(df, genders):
    df = df[df['sexe'].isin(genders)]
    return df

In [9]:
def df_filter_departments(df, departments):
    df = df[df['dpt'].isin(departments)]
    return df

In [10]:
def df_filter_years(df, year_start=None, year_end=None):
    if year_start is not None:
        df = df[df['annais'] >= year_start]
        
    if year_end is not None:
        df = df[df['annais'] <= year_end]
        
    return df

In [11]:
def df_groupy(df, columns):
    df = df.groupby(columns, as_index=False)['nombre'].sum()
    return df

In [12]:
def get_top(df, n=10):
    df = df.sort_values('nombre', ascending=False).reset_index(drop=True).head(10)
    return df

In [13]:
def get_genders(genders):
    if genders == 'Boys':
        return ['M']
    elif genders == 'Girls':
        return ['F']
    else:
        return ['M', 'F']

In [ ]:
from dash import Dash, html, dcc, Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

print(df['annais'].min(), df['annais'].max())

app.layout = html.Div([
    
    html.Div([
        dcc.Dropdown(
            ['Boys and Girls', 'Boys', 'Girls'],
            'Boys and Girls',
            id='gender-selection'
        ),
    ], style={'width': '24%', 'display': 'inline-block'}),
    
    
    html.Div([
        dcc.Input(
            type='text',
            value=None,
            placeholder='Filter names',
            id='input_name'  
        )
    ], style={'width': '14%', 'display': 'inline-block', 'vertical-align':'top'}),
    
    html.Div([
        dcc.RadioItems(
            ['Input: Map', 'Input: Barchart'], 
            'Input: Map', 
            inline=True,
            id='input-choice'
        )
    ], style={'width': '14%', 'display': 'inline-block', 'vertical-align':'top'}),
    
    html.Div([
        dcc.RangeSlider(
            min=min_year, max=max_year, step=10,
            value = [min_year, max_year],
            marks=marks_year,
            id='year-slider'
        )
    ], style={'width': '39%', 'display': 'inline-block'}), 
    
    html.Div([
        dcc.Graph(
            id='map'
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.Graph(
            id='bar'
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
])


@app.callback(
    Output('map', 'figure'), 
    Input('gender-selection', 'value'), 
    Input('input_name', 'value'),
    Input('year-slider', 'value'),
    Input('input-choice', 'value'),
    Input('bar', 'selectedData'))
def update_map(gender_choice, input_name, pair_years, choice_input, selectedData):
    df_copy = df.copy()
    
    genders = get_genders(gender_choice)
    
    df_copy = df_filter_genders(df_copy, genders=genders)
    df_copy = df_filter_years(df_copy, year_start=pair_years[0], year_end=pair_years[1])
    
    if input_name is not None:
        df_copy = df_filter_partial_name(df_copy, partial_name=input_name.upper())
    
    if choice_input == 'Input: Barchart':
        df_copy = df_groupy(df_copy, columns=['preusuel', 'sexe', 'dpt'])

        if selectedData is not None:
            gendered_names = []
            for point in selectedData['points']:
                gendered_name = (point['x'], point['customdata'][0])
                gendered_names.append(gendered_name)
            df_copy = df_filter_gendered_names(df_copy, gendered_names=gendered_names)
        
    
    df_copy = df_groupy(df_copy, columns=['dpt'])
    
    fig = px.choropleth_mapbox(df_copy, geojson=geojson_departments, 
                               locations='dpt', 
                               color=df_copy['nombre'], 
                               color_continuous_scale='Turbo',
                               mapbox_style='carto-positron',
                               zoom=4, 
                               center = {"lat": 46, "lon": 2},
                               opacity=0.5)

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    if choice_input == 'Input: Map':
        fig.update_layout(clickmode='event+select')
    
    return fig

@app.callback(
    Output('bar', 'figure'), 
    Input('gender-selection', 'value'),
    Input('input_name', 'value'),
    Input('year-slider', 'value'),
    Input('input-choice', 'value'),
    Input('map', 'selectedData'))
def update_bar(gender_choice, input_name, pair_years, choice_input, selectedData):
    df_copy = df.copy()
    genders = get_genders(gender_choice)
    
    df_copy = df_filter_genders(df_copy, genders=genders)
    df_copy = df_filter_years(df_copy, year_start=pair_years[0], year_end=pair_years[1])
    
    if input_name is not None:
        df_copy = df_filter_partial_name(df_copy, partial_name=input_name.upper())
    
    if choice_input == 'Input: Map':
        if selectedData is not None:
            locations = []
            for points in selectedData['points']:
                location = points['location']
                locations.append(location)   
            df_copy = df_filter_departments(df_copy, departments=locations)
    
    df_copy = df_groupy(df_copy, columns=['preusuel', 'sexe'])
    df_copy = get_top(df_copy, n=10)
    
    fig = px.bar(df_copy, 
                 x='preusuel', 
                 y='nombre', 
                 color='sexe', 
                 custom_data=['sexe'],
                 color_discrete_map={'F':'gold', 'M':'darkblue'})
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, xaxis={'categoryorder':'total descending'})
    if choice_input == 'Input: Barchart':
        fig.update_layout(clickmode='event+select')
    
    return fig

app.run_server()

1900 2020
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jun/2022 22:28:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2022 22:28:20] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Jun/2022 22:28:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2022 22:28:26] "POST /_dash-